In [7]:
from loguru import logger
from tqdm import tqdm
import pickle
import json

In [8]:
original_data_path = "/disk1/data/ing_mlm_data/original"

# log
logger.info(f"original_file_path = {original_data_path}")

with open(f"{original_data_path}/iids_full.pkl", "rb") as f:
    iids_data = pickle.load(f)

with open(f"{original_data_path}/iid2ingr_full.pkl", "rb") as f:
    iid2ingr_data = pickle.load(f)

with open(f"{original_data_path}/iid2fgvec_full.pkl", "rb") as f:
    iids2fgvec_data = pickle.load(f)

with open(f"{original_data_path}/rid2title.pkl", "rb") as f:
    rid2title_data = pickle.load(f)

# split info
with open(f"{original_data_path}/rid2info_full.pkl", "rb") as f:
    rid2info_full = pickle.load(f)

with open(f"{original_data_path}/rid2sorted_iids_full.pkl", "rb") as f:
    rid2sorted_iids_data = pickle.load(f)

with open(f"{original_data_path}/tags.json", "rb") as f:
    rid2tags = json.load(f)

2024-03-19 23:46:45.076 | INFO     | __main__:<module>:4 - original_file_path = /disk1/data/ing_mlm_data/original


In [28]:
import pandas as pd

food = pd.read_csv('/disk1/data/Food.com/RAW_recipes.csv')
food.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [473]:
print(food['steps'][2])
print(food['ingredients'][0])


['brown ground beef in large pot', 'add chopped onions to ground beef when almost brown and sautee until wilted', 'add all other ingredients', 'add kidney beans if you like beans in your chili', 'cook in slow cooker on high for 2-3 hours or 6-8 hours on low', 'serve with cold clean lettuce and shredded cheese']
['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']


In [10]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Dizex/FoodBaseBERT")
model = AutoModelForTokenClassification.from_pretrained("Dizex/FoodBaseBERT")

pipe = pipeline("ner", model=model, tokenizer=tokenizer)


In [52]:
## test code
ner_entity_results = pipe(eval(food['steps'][1])[4])
print(ner_entity_results)
print('')

for i, v in enumerate(ner_entity_results):
    print(v)
    print(v['word'])

[{'entity': 'B-FOOD', 'score': 0.9972826, 'index': 4, 'word': 'eggs', 'start': 6, 'end': 10}, {'entity': 'B-FOOD', 'score': 0.99657214, 'index': 6, 'word': 'milk', 'start': 15, 'end': 19}]

{'entity': 'B-FOOD', 'score': 0.9972826, 'index': 4, 'word': 'eggs', 'start': 6, 'end': 10}
eggs
{'entity': 'B-FOOD', 'score': 0.99657214, 'index': 6, 'word': 'milk', 'start': 15, 'end': 19}
milk


In [265]:
from tqdm import tqdm
recipe_seq = []
# cnt = 0
for steps in tqdm(food['steps'][:10000]):
    steps = eval(steps)
    # print(steps)
    recipe_seq_tmp2 = []
    for step in steps:
        ner_entity_results = pipe(step)
        # print(ner_entity_results)
        recipe_seq_tmp1 = []
        for v in ner_entity_results:
            # print(v['word'])
            recipe_seq_tmp1.append(v['word'])
        recipe_seq_tmp2.append(recipe_seq_tmp1)
        # print('')
    recipe_seq.append(recipe_seq_tmp2)
    # cnt +=1
    # if cnt > 1 : break



100%|██████████| 10000/10000 [1:16:54<00:00,  2.17it/s]


In [267]:
# temp save
recipe_seq_save = [str(n) for n in recipe_seq]
import pandas as pd
recipe_seq_df = pd.DataFrame(recipe_seq_save)
recipe_seq_df.to_csv("/home/jaesung/jaesung/research/recipetrend/preprocess/recipe_seq_10K")

In [268]:
print(recipe_seq[0])
print(recipe_seq[1])

[[], ['squash'], ['seeds'], ['s', '##pic', '##y', 'squash', 'olive', 'oil', 'melted', 'butter', 'cut', 'squash', 'piece'], ['me', '##xi', '##can', 'season', '##ing', 'mix'], ['sweet', 'squash', 'melted', 'honey', 'butter', 'g', '##rated', 'p', '##ilon', '##ci', '##llo', 'cut', 'squash'], ['sweet', 'me', '##xi', '##can', 's', '##pice'], ['skin'], ['squash', 'especially', 'sugar', 'butter'], ['squash'], ['salt']]
[[], ['dough'], [], ['sa', '##usage'], ['eggs', 'milk'], ['sa', '##usage', 'baked', 'crust', 'cheese'], ['egg', 'mixture', 'sa', '##usage', 'cheese'], [], ['eggs', 'crust']]


In [269]:
refac_recipe_seq = []
for seq in recipe_seq:
    refac_recipe_seq.append([n for n in seq if len(n)!=0])

In [270]:
print(refac_recipe_seq[0])
print(refac_recipe_seq[1])

[['squash'], ['seeds'], ['s', '##pic', '##y', 'squash', 'olive', 'oil', 'melted', 'butter', 'cut', 'squash', 'piece'], ['me', '##xi', '##can', 'season', '##ing', 'mix'], ['sweet', 'squash', 'melted', 'honey', 'butter', 'g', '##rated', 'p', '##ilon', '##ci', '##llo', 'cut', 'squash'], ['sweet', 'me', '##xi', '##can', 's', '##pice'], ['skin'], ['squash', 'especially', 'sugar', 'butter'], ['squash'], ['salt']]
[['dough'], ['sa', '##usage'], ['eggs', 'milk'], ['sa', '##usage', 'baked', 'crust', 'cheese'], ['egg', 'mixture', 'sa', '##usage', 'cheese'], ['eggs', 'crust']]


In [273]:
preprocessed_recipe_seq = []
# cnt = 0
for seq in tqdm(refac_recipe_seq):
    preprocessed_recipe_seq_temp2 = []
    for s in seq:
        # print(s)
        preprocessed_recipe_seq_temp1 = []
        for ss in s:
            # print(ss)
            tt = tokenizer(ss)
            # rint(tt['input_ids'])
            for t in tt['input_ids']:
                if (t != 0) and (t != 101) and (t != 102) and (t != 103):
                    # print(t)
                    preprocessed_recipe_seq_temp1.append(t)
        preprocessed_recipe_seq_temp2.append(preprocessed_recipe_seq_temp1)
        # print('')
    # print(seq)
    preprocessed_recipe_seq.append(preprocessed_recipe_seq_temp2)
    # print('')
    # cnt += 1
    # if cnt > 1 : break

100%|██████████| 10000/10000 [00:07<00:00, 1381.37it/s]


In [274]:
print(preprocessed_recipe_seq[0])
print(preprocessed_recipe_seq[1])

[[18794], [8079], [1055, 1001, 1001, 27263, 1001, 1001, 1061, 18794, 9724, 3514, 12501, 12136, 3013, 18794, 3538], [2033, 1001, 1001, 8418, 1001, 1001, 2064, 2161, 1001, 1001, 13749, 4666], [4086, 18794, 12501, 6861, 12136, 1043, 1001, 1001, 6758, 1052, 1001, 1001, 6335, 2239, 1001, 1001, 25022, 1001, 1001, 2222, 2080, 3013, 18794], [4086, 2033, 1001, 1001, 8418, 1001, 1001, 2064, 1055, 1001, 1001, 27263, 2063], [3096], [18794, 2926, 5699, 12136], [18794], [5474]]
[[23126], [7842, 1001, 1001, 8192], [6763, 6501], [7842, 1001, 1001, 8192, 17776, 19116, 8808], [8288, 8150, 7842, 1001, 1001, 8192, 8808], [6763, 19116]]


In [275]:
# cnt = 0
tmp = []
for seq_id in preprocessed_recipe_seq:
    # print(seq_id)
    tmp1 = []
    for seq in seq_id :
        # print(seq)

        final_recipe_seq = tokenizer.decode(seq)
        tmp1.append(final_recipe_seq)
    tmp.append(tmp1)
    # cnt += 1
    # if cnt > 1 : break

In [276]:
# cnt = 0
temp = []
for t in tmp:
    # print(t)
    temp1 = []
    for tt in t :
        # print(tt.replace(' # # ' ,''))
        temp1.append(tt.replace(' # # ', ''))
    temp.append(temp1)

    # cnt += 1
    # if cnt > 1 :break

In [279]:
temp = [str(n) for n in temp]

temp_df = pd.DataFrame(temp)
temp_df.to_csv('/home/jaesung/jaesung/research/recipetrend/preprocess/recipe_seq_10K.csv')

In [ ]:
## finish

In [287]:
f10k = pd.concat([food[:10000],temp_df], axis=1)
f10k.rename(columns={0:'sequence'}, inplace=True)

In [474]:
f10k['new_sequence'][0]

[[0, 'winter squash'],
 [2, 'olive oil'],
 [3, 'mexican seasoning'],
 [5, 'mixed spice'],
 [6, 'honey'],
 [7, 'butter'],
 [9, 'salt']]

In [294]:
print(f10k['ingredients'][0])
print(f10k['sequence'][0])
print(type(f10k['ingredients'][0]))
print(type(f10k['sequence'][0]))

['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']
['squash', 'seeds', 'spicy squash olive oil melted butter cut squash piece', 'mexican seasoning mix', 'sweet squash melted honey butter grated piloncillo cut squash', 'sweet mexican spice', 'skin', 'squash especially sugar butter', 'squash', 'salt']
<class 'str'>
<class 'str'>


In [292]:
import difflib

In [314]:
cnt = 0

new_seq = []
for j in range(10000):
    new_seq_tmp = []
    for _set in eval(f10k['ingredients'][j]):
        max = -1000; most_sim = ''; most_sim_idx = 0
        for i, _seq in enumerate(eval(f10k['sequence'][j])):
            # print(_set)
            # print(_seq)
            answer_bytes = bytes(_set, 'utf-8')
            input_bytes = bytes(_seq, 'utf-8')
            answer_bytes_list = list(answer_bytes)
            input_bytes_list = list(input_bytes)

            sm = difflib.SequenceMatcher(None, answer_bytes_list, input_bytes_list)
            similar = sm.ratio()

            if similar > max:
                max = similar
                most_sim = _seq
                most_sim_idx = i
            # print(similar)
        new_seq_tmp.append([most_sim_idx, _set])
        # print(most_sim)
        #print(max)
        # print(most_sim_idx)
    new_seq.append(new_seq_tmp)

        # cnt += 1
        # if cnt > 0 : break


In [322]:
for v in new_seq:
    v.sort(key=lambda a: a[0])

In [327]:
f10k['new_sequence'] = new_seq

In [328]:
f10k.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,sequence,new_sequence
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,"['squash', 'seeds', 'spicy squash olive oil me...","[[0, winter squash], [2, olive oil], [3, mexic..."
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,"['dough', 'sausage', 'eggs milk', 'sausage bak...","[[1, sausage patty], [1, salt and pepper], [2,..."
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,"['ground beef', 'chopped onions ground beef', ...","[[0, ground beef], [0, tomato paste], [0, rote..."
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,"['potatoes salted water', 'potatoes', 'potatoe...","[[0, spreadable cheese with garlic and herbs],..."
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,['ketchup'],"[[0, tomato juice], [0, apple cider vinegar], ..."


In [ ]:
## to make txt file

In [ ]:
## set

In [330]:
f10k_set = f10k['ingredients']

In [402]:
cnt = 0
tmp = []
for v in f10k_set:
    tmp1 = []
    for u in eval(v):
        u = u.replace(' ', '_')
        tmp1.append(u)
    tmp.append(tmp1)

    

In [403]:
cnt = 0 

temp = []
for t in tmp:
    temp.append(' '.join(t))

In [416]:
tn = f10k[['tags', 'name']]

tn['tags'] = tn['tags'].apply(lambda x: eval(x))
tn['tags'] = tn['tags'].apply(lambda x: ' '.join(x))

/tmp/ipykernel_3480816/3894532314.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tn['tags'] = tn['tags'].apply(lambda x: eval(x))
/tmp/ipykernel_3480816/3894532314.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tn['tags'] = tn['tags'].apply(lambda x: ' '.join(x))


In [417]:
tn.insert(0, 'recipe_set', temp)

In [419]:
tn['new_set'] = tn['recipe_set'] + "[SEP]" + tn['tags'] + "[SEP]" + tn['name']

/tmp/ipykernel_3480816/3861825294.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tn['new_set'] = tn['recipe_set'] + "[SEP]" + tn['tags'] + "[SEP]" + tn['name']


In [432]:
tn['new_set'] = tn['new_set'].apply(lambda x: str(x))

/tmp/ipykernel_3480816/3544343899.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tn['new_set'] = tn['new_set'].apply(lambda x: str(x))


In [434]:
# train
f = open('/home/jaesung/jaesung/research/recipetrend/preprocess/f10k_set_train.txt', '+w')

for v in tn['new_set'][:9000]:
    f.write(v+'\n')


In [435]:
# val
f = open('/home/jaesung/jaesung/research/recipetrend/preprocess/f10k_set_val.txt', '+w')

for v in tn['new_set'][9000:]:
    f.write(v+'\n')

In [ ]:
## seq

In [345]:
f10k_seq = f10k['new_sequence']

In [444]:
cnt = 0
tmp = []
for v in f10k_seq:
    tmp1 = []
    for u in v:
        tmp1.append(u[1])
    tmp.append(tmp1)
    # cnt += 1
    # if cnt > 3 : break

In [445]:
tmp2 = []
for v in tmp:
    tmp1 = []
    for u in v:
        u = u.replace(' ', '_')
        tmp1.append(u)
    tmp2.append(tmp1)


In [461]:
cnt = 0 

temp = []
for t in tmp2:
    temp.append(' '.join(t))

In [462]:
tn2 = f10k[['tags', 'name']]

tn2['tags'] = tn2['tags'].apply(lambda x: eval(x))
tn2['tags'] = tn2['tags'].apply(lambda x: ' '.join(x))

/tmp/ipykernel_3480816/3814154758.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tn2['tags'] = tn2['tags'].apply(lambda x: eval(x))
/tmp/ipykernel_3480816/3814154758.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tn2['tags'] = tn2['tags'].apply(lambda x: ' '.join(x))


In [464]:

tn2.insert(0, 'recipe_seq', temp)

In [465]:

tn2['new_seq'] = tn2['recipe_seq'] + "[SEP]" + tn2['tags'] + "[SEP]" + tn2['name']

/tmp/ipykernel_3480816/1385527983.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tn2['new_seq'] = tn2['recipe_seq'] + "[SEP]" + tn2['tags'] + "[SEP]" + tn2['name']


In [467]:

tn2['new_seq'] = tn2['new_seq'].apply(lambda x: str(x))

/tmp/ipykernel_3480816/3574774114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tn2['new_seq'] = tn2['new_seq'].apply(lambda x: str(x))


In [469]:
tn2['new_seq'][0]

'winter_squash olive_oil mexican_seasoning mixed_spice honey butter salt[SEP]60-minutes-or-less time-to-make course main-ingredient cuisine preparation occasion north-american side-dishes vegetables mexican easy fall holiday-event vegetarian winter dietary christmas seasonal squash[SEP]arriba   baked winter squash mexican style'

In [470]:
# train
f = open('/home/jaesung/jaesung/research/recipetrend/preprocess/f10k_seq_train.txt', '+w')

for v in tn2['new_seq'][:9000]:
    f.write(v+'\n')

In [471]:
# val
f = open('/home/jaesung/jaesung/research/recipetrend/preprocess/f10k_seq_val.txt', '+w')

for v in tn2['new_seq'][9000:]:
    f.write(v+'\n')

In [17]:
import pandas as pd 

food = pd.read_csv('/home/jaesung/jaesung/research/recipetrend/preprocess/RecipeNLG_dataset.csv')
food.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [27]:
k = 8
print(food['ingredients'][k])
print(food['directions'][k])
print(food['NER'][k])

["1 1/2 lb. round steak (1-inch thick), cut into strips", "1 can drained tomatoes, cut up (save liquid)", "1 3/4 c. water", "1/2 c. onions", "1 1/2 Tbsp. Worcestershire sauce", "2 green peppers, diced", "1/4 c. oil"]
["Roll steak strips in flour.", "Brown in skillet.", "Salt and pepper.", "Combine tomato liquid, water, onions and browned steak. Cover and simmer for one and a quarter hours.", "Uncover and stir in Worcestershire sauce.", "Add tomatoes, green peppers and simmer for 5 minutes.", "Serve over hot cooked rice."]
["tomatoes", "water", "onions", "Worcestershire sauce", "green peppers", "oil"]
